In [13]:
import os
import pandas as pd
from selenium import webdriver
import selenium.webdriver.chrome.options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [2]:
round_ = '19'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-19'

In [3]:
day = datetime.now().strftime("%A").lower()
day

'saturday'

In [4]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [5]:
tmp = soup.find_all('script')

In [6]:
game_urls = []
for i in tmp[:9]:
    try:
        url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
        game_url = url_find.search(i.text)
        game_urls.append(game_url.group())
    except:
        continue
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/richmond-v-hawthorn-7485783',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/carlton-v-west-coast-7485784',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/brisbane-v-geelong-7485785',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/fremantle-v-sydney-7485786',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/port-adelaide-v-collingwood-7485797',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/greater-western-sydney-v-gold-coast-7485799',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/melbourne-v-adelaide-7485800',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/st-kilda-v-north-melbourne-7485801']

In [7]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [8]:
game_list

[[('richmond', 'hawthorn')],
 [('carlton', 'west-coast')],
 [('brisbane', 'geelong')],
 [('fremantle', 'sydney')],
 [('port-adelaide', 'collingwood')],
 [('greater-western-sydney', 'gold-coast')],
 [('melbourne', 'adelaide')],
 [('st-kilda', 'north-melbourne')]]

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

ValueError: There is no such driver by url https://chromedriver.storage.googleapis.com/LATEST_RELEASE_115.0.5790

In [27]:
disposals = ['To Get 15 or More Disposals',
            'To Get 20 or More Disposals',
            'To Get 25 or More Disposals',
            'To Get 30 or More Disposals',]
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [28]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
        
        
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,50):
            try:
                tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
            except:
                continue
            if tmp in disposals:

                dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

                dis.click()

                player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
                print(player)
                new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
                output.loc[len(output)] = new_row

                for j in range(1,50):
                    try:

                        tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                        print(tmp1)
                        new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                        output.loc[len(output)] = new_row
                    except:
                        continue
                
                dis.click()
                time.sleep(2)
            
    except:
        continue

['Jordan De Goey', '1.06']
['John Noble', '1.07']
['Jack Crisp', '1.13']
['Steele Sidebottom', '1.14']
['Isaac Quaynor', '1.19']
['Patrick Lipinski', '1.22']
['Brayden Maynard', '1.25']
['Taylor Adams', '1.26']
['Jeremy Howe', '1.32']
['Darcy Moore', '1.36']
['Daniel McStay', '2.50']
['Mason Cox', '2.80']
['Billy Frampton', '2.95']
['Jamie Elliott', '4.00']
['Ash Johnson', '4.70']
['Luke Ryan', '1.09']
['Liam Henry', '1.10']
['Jordan Clark', '1.12']
['James Aish', '1.15']
['Sean Darcy', '1.34']
['Matt Johnson', '1.38']
['Lachie Schultz', '1.40']
['Neil Erasmus', '1.49']
['Luke Jackson', '1.55']
['Corey Wagner', '1.68']
['Bailey Banfield', '2.35']
['Sam Switkowski', '2.35']
['Michael Walters', '4.60']
['Josh Daicos', '1.17']
['Scott Pendlebury', '1.27']
['Jordan De Goey', '1.30']
['John Noble', '1.49']
['Steele Sidebottom', '1.57']
['Jack Crisp', '1.73']
['Isaac Quaynor', '1.96']
['Taylor Adams', '2.20']
['Brayden Maynard', '2.25']
['Patrick Lipinski', '2.25']
['Darcy Moore', '2.70']
['

['Jacob Weitering', 'Last 5: 11-17-16-15-16', '1.62']
['Lachlan Fogarty', 'Last 5: 17-18-10-0-10', '1.93']
['Charlie Curnow', 'Last 5: 15-13-15-17-11', '1.96']
['Harry McKay', 'Last 5: 20-19-10-14-12', '2.25']
['Alex Cincotta', 'Last 5: 11-11-11-18-10', '2.35']
['Jordan Boyd', 'Last 5: 10-14-11-14-9', '2.35']
['Tom De Koning', 'Last 5: 11-12-8-13-9', '3.00']
['Jack Martin', 'Last 5: 14-12-8-8-6', '3.90']
['Dylan Williams', 'Last 5: 17-22-19-16-19', '1.31']
['Travis Boak', 'Last 5: 20-8-13-26-12', '1.32']
['Xavier Duursma', 'Last 5: 5-19-11-22-8', '1.41']
['Willem Drew', 'Last 5: 14-18-14-14-17', '1.43']
['Kane Farrell', 'Last 5: 17-25-15-17-14', '1.47']
['Jeremy Finlayson', 'Last 5: 14-14-10-8-17', '1.69']
['Darcy Byrne-Jones', 'Last 5: 17-8-14-7-12', '1.90']
['Lachlan Jones', 'Last 5: N/A', '2.12']
['Miles Bergman', 'Last 5: 10-7-6-10-17', '2.25']
['Sam Powell-Pepper', 'Last 5: 12-12-13-10-18', '2.30']
['Scott Lycett', 'Last 5: 9-11-13-18-13', '2.50']
['Adam Cerra', 'Last 5: 27-27-27-

['Archie Perkins', 'Last 5: 24-12-16-18-18', '6.25']
['Jake Stringer', 'Last 5: 8-13-16-18-17', '17.00']
['Patrick Dangerfield', 'Last 5: 26-15-15-16-28', '3.90']
['Tom Stewart', 'Last 5: 26-25-22-18-27', '4.25']
['Mitch Duncan', 'Last 5: 30-25-22-20-25', '5.10']
['Gryan Miers', 'Last 5: 32-27-15-13-20', '10.50']
['Isaac Smith', 'Last 5: 18-17-20-22-24', '14.00']
['Jack Bowes', 'Last 5: 25-3-12-5-3', '14.00']
['Max Holmes', 'Last 5: 26-8-17-24-13', '14.00']
['Tanner Bruhn', 'Last 5: 22-18-19-12-15', '26.00']
['Tom Atkins', 'Last 5: 25-18-19-12-19', '26.00']
['Darcy Parish', 'Last 5: 39-29-24-37-38', '1.50']
['Zach Merrett', 'Last 5: 39-31-29-18-34', '1.70']
['Nic Martin', 'Last 5: 28-16-26-30-21', '5.60']
['Andrew McGrath', 'Last 5: 31-20-27-21-19', '7.00']
['Ben Hobbs', 'Last 5: 20-23-26-24-21', '7.00']
['Dyson Heppell', 'Last 5: 25-20-24-23-20', '9.25']
['Mason Redman', 'Last 5: 19-21-13-23-19', '10.50']
['Jye Caldwell', 'Last 5: 20-24-21-25-27', '12.50']
['Mitch Hinge', 'Last 5: 27-

['Jai Newcombe', 'Last 5: 28-21-20-28-19', '1.19']
['Jarman Impey', 'Last 5: 7-29-28-33-26', '1.30']
['Karl Amon', 'Last 5: 27-19-18-20-25', '1.35']
['Dylan Moore', 'Last 5: 13-15-17-25-22', '1.62']
['Blake Hardwick', 'Last 5: 25-12-38-22-25', '1.70']
['Josh Ward', 'Last 5: 20-28-15-20-20', '1.71']
['Josh Weddle', 'Last 5: 20-23-10-18-25', '1.98']
['Jack Scrimshaw', 'Last 5: 17-18-31-19-19', '2.20']
['Connor Macdonald', 'Last 5: 23-15-13-28-20', '2.25']
['Chad Wingard', 'Last 5: 16-18-8-12-17', '3.60']
['Luke Davies-Uniacke', 'Last 5: 28-22-29-25-20', '1.54']
['Harry Sheezel', 'Last 5: 15-31-32-24-21', '1.63']
['Tarryn Thomas', 'Last 5: 24-23-24-19-17', '3.10']
['Bailey Scott', 'Last 5: 24-24-20-23-19', '3.25']
['Liam Shiels', 'Last 5: N/A', '4.10']
['George Wardlaw', 'Last 5: 22-9-22-17-12', '4.90']
['Luke McDonald', 'Last 5: 25-21-15-14-18', '5.10']
['Cameron Zurhaar', 'Last 5: 25-14-14-10-12', '8.00']
['Tristan Xerri', 'Last 5: 19-5-5-14-14', '8.50']
['Darcy Tucker', 'Last 5: 12-22-

In [29]:
output

,disposals,round,teams,Player,odds
0,To Get 15 or More Disposals,R18,collingwood v fremantle,jordan de goey,1.06
1,To Get 15 or More Disposals,R18,collingwood v fremantle,john noble,1.07
2,To Get 15 or More Disposals,R18,collingwood v fremantle,jack crisp,1.13
3,To Get 15 or More Disposals,R18,collingwood v fremantle,steele sidebottom,1.14
4,To Get 15 or More Disposals,R18,collingwood v fremantle,isaac quaynor,1.19
...,...,...,...,...,...
693,To Get 30 or More Disposals,R18,west-coast v richmond,nick vlastuin,11.50
694,To Get 30 or More Disposals,R18,west-coast v richmond,jack ross,21.00
695,To Get 30 or More Disposals,R18,west-coast v richmond,liam baker,26.00
696,To Get 30 or More Disposals,R18,west-coast v richmond,noah balta,26.00


In [30]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [31]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'north-melbourne':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [32]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [33]:
output

,disposals,round,teams,Player,odds,home,away
0,To Get 15 or More Disposals,R18,collingwood v fremantle,jordan de goey,1.06,COL,FRE
1,To Get 15 or More Disposals,R18,collingwood v fremantle,john noble,1.07,COL,FRE
2,To Get 15 or More Disposals,R18,collingwood v fremantle,jack crisp,1.13,COL,FRE
3,To Get 15 or More Disposals,R18,collingwood v fremantle,steele sidebottom,1.14,COL,FRE
4,To Get 15 or More Disposals,R18,collingwood v fremantle,isaac quaynor,1.19,COL,FRE
...,...,...,...,...,...,...,...
693,To Get 30 or More Disposals,R18,west-coast v richmond,nick vlastuin,11.50,WCE,RCH
694,To Get 30 or More Disposals,R18,west-coast v richmond,jack ross,21.00,WCE,RCH
695,To Get 30 or More Disposals,R18,west-coast v richmond,liam baker,26.00,WCE,RCH
696,To Get 30 or More Disposals,R18,west-coast v richmond,noah balta,26.00,WCE,RCH


In [34]:
output.to_csv(f'/Users/caseyh/Desktop/footyfootyfooty/sportsbet_odds/r{round_}/disposal_markets_{day}.csv', index = False)